In [ ]:
df_model = df_player[['teamConf', 'teamLoc', 'teamRslt', 'playStat', 'playMin', 'playPos', 'playHeight', 'playWeight', 'playPTS', 'playAST', 'playTO', 'playSTL', 'playBLK', 'playPF', 'playFGA', 'playFGM', 'playFG%', 'play2PA', 'play2PM', 'play2P%', 'play3PA', 'play3PM', 'play3P%', 'playFTA', 'playFTM', 'playFT%', 'playORB', 'playDRB', 'playTRB', 'opptConf', 'sub_team', 'is_allnba', 'age']]
#df_model = df_player[['teamConf', 'teamLoc', 'teamRslt', 'playStat', 'playPos', 'playHeight', 'playWeight', 'playFIC40', 'opptConf', 'sub_team', 'is_allnba', 'age']]

In [ ]:
# change sub_team 1230 to 3210
#df_model['sub_team'] = df_player['sub_team'].replace([1, 2, 3, 0], [3, 2, 1, 0])

# change sub_team 1230 to first_team, second_team, third_team, isnt_all_nba
df_model['sub_team'] = df_player['sub_team'].replace([1, 2, 3, 0], ['first_team', 'second_team', 'thrid_team', 'isnt_allnba'])

In [ ]:
# Cell for one hot encoding creation
teamconf_ohe = pd.get_dummies(df_model['teamConf'])
teamconf_ohe.rename(columns={"East": "team_east", "West": "team_west"}, inplace=True)
teamLoc_ohe = pd.get_dummies(df_model['teamLoc'])
teamRslt_ohe = pd.get_dummies(df_model['teamRslt'])
playStat_ohe = pd.get_dummies(df_model['playStat'])
opptConf_ohe = pd.get_dummies(df_model['opptConf'])
opptConf_ohe.rename(columns={"East": "oppt_east", "West": "oppt_west"}, inplace=True)
playPos_ohe = pd.get_dummies(df_model['playPos'])

#df_model = pd.concat([df_model,teamconf_ohe,teamLoc_ohe,teamRslt_ohe,playStat_ohe,opptConf_ohe,playPos_ohe], axis=1)
#df_model = df_model.drop(columns=['teamConf','teamLoc','teamRslt','playStat','opptConf','playPos'])

# one-hot encoding sub_team
sub_team_ohe = pd.get_dummies(df_model['sub_team'])
df_model = pd.concat([df_model,teamconf_ohe,teamLoc_ohe,teamRslt_ohe,playStat_ohe,opptConf_ohe,playPos_ohe,sub_team_ohe], axis=1)
df_model = df_model.drop(columns=['teamConf','teamLoc','teamRslt','playStat','opptConf','playPos','sub_team'])

In [ ]:
df_model.isna().sum()

In [ ]:
# Train Test split
np.random.seed(47)

#X = df_model.drop(columns=['is_allnba', 'sub_team'])
#Y = pd.Series(df_model['sub_team'])

# one-hot encoding sub_team
X = df_model.drop(columns=['is_allnba', 'first_team', 'second_team', 'thrid_team', 'isnt_allnba'])
#Y = pd.Series(df_model['first_team'])
#Y = pd.Series(df_model['second_team'])
#Y = pd.Series(df_model['thrid_team'])
#Y = pd.Series(df_model['isnt_allnba'])
Y = pd.Series(df_model['is_allnba'])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
# Random Forest Model
random_forest_model = ensemble.RandomForestClassifier(n_estimators=65)
random_forest_model.fit(X_train, Y_train)
Y_predict = random_forest_model.predict(X_test)
Y_fitted = random_forest_model.predict(X_train)

In [ ]:
# all_nba precision, recall, false-alarm
training_TP = 0
training_FP = 0
training_FN = 0
training_TN = 0

testing_TP = 0
testing_FP = 0
testing_FN = 0
testing_TN = 0

Y_train_array = Y_train.to_numpy()
Y_test_array = Y_test.to_numpy()

for i in range(len(X_train)):
    if Y_fitted[i] == 1 and Y_train_array[i] == 1:
        training_TP += 1
    elif Y_fitted[i] == 1 and Y_train_array[i] == 0:
        training_FP += 1
    elif Y_fitted[i] == 0 and Y_train_array[i] == 1:
        training_FN += 1
    elif Y_fitted[i] == 0 and Y_train_array[i] == 0:
        training_TN += 1

for i in range(len(X_test)):
    if Y_predict[i] == 1 and Y_test_array[i] == 1:
        testing_TP += 1
    elif Y_predict[i] == 1 and Y_test_array[i] == 0:
        testing_FP += 1
    elif Y_predict[i] == 0 and Y_test_array[i] == 1:
        testing_FN += 1
    elif Y_predict[i] == 0 and Y_test_array[i] == 0:
        testing_TN += 1

training_precision = training_TP / (training_TP + training_FP)
training_recall = training_TP / (training_TP + training_FN)
training_far = training_FP / (training_FP + training_TN)
training_f_measure = 2.0 * training_precision * training_recall / (training_precision + training_recall)

testing_precision = testing_TP / (testing_TP + testing_FP)
testing_recall = testing_TP / (testing_TP + testing_FN)
testing_far = testing_FP / (testing_FP + testing_TN)
testing_f_measure = 2.0 * testing_precision * testing_recall / (testing_precision + testing_recall)

training_accuracy = sum(Y_train == Y_fitted) / len(Y_fitted)
testing_accuracy = sum(Y_test == Y_predict) / len(Y_test)

print("Training Accuracy:", training_accuracy)
print("Testing Accuracy:", testing_accuracy)

print("Training Precision:", training_precision)
print("Training Recall:", training_recall)
print("Training False-alarm:", training_far)
print("Training F-Measure:", training_f_measure)

print("Testing Precision:", testing_precision)
print("Testing Recall:", testing_recall)
print("Testing False-alarm:", testing_far)
print("Testing F-Measure:", testing_f_measure)

'''
# sub_team precision, recall, false-alarm
first_TP = 0
first_FP = 0
first_FN = 0
first_TN = 0
second_TP = 0
second_FP = 0
second_FN = 0
second_TN = 0
third_TP = 0
third_FP = 0
third_FN = 0
third_TN = 0

Y_train_array = Y_train.to_numpy()

for i in range(len(X_train)):
    if Y_fitted[i] == 3 and Y_train_array[i] == 3:
        first_TP += 1
    elif Y_fitted[i] == 3 and Y_train_array[i] != 3:
        first_FP += 1
    elif Y_fitted[i] != 3 and Y_train_array[i] == 3:
        first_FN += 1
    elif Y_fitted[i] != 3 and Y_train_array[i] != 3:
        first_TN += 1

for i in range(len(X_train)):
    if Y_fitted[i] == 2 and Y_train_array[i] == 2:
        second_TP += 1
    elif Y_fitted[i] == 2 and Y_train_array[i] != 2:
        second_FP += 1
    elif Y_fitted[i] != 2 and Y_train_array[i] == 2:
        second_FN += 1
    elif Y_fitted[i] != 2 and Y_train_array[i] != 2:
        second_TN += 1

for i in range(len(X_train)):
    if Y_fitted[i] == 1 and Y_train_array[i] == 1:
        third_TP += 1
    elif Y_fitted[i] == 1 and Y_train_array[i] != 1:
        third_FP += 1
    elif Y_fitted[i] != 1 and Y_train_array[i] == 1:
        third_FN += 1
    elif Y_fitted[i] != 1 and Y_train_array[i] != 1:
        third_TN += 1

first_logistic_predictor_precision = first_TP / (first_TP + first_FP)
first_logistic_predictor_recall = first_TP / (first_TP + first_FN)
first_logistic_predictor_far = first_FP / (first_FP + first_TN)

second_logistic_predictor_precision = second_TP / (second_TP + second_FP)
second_logistic_predictor_recall = second_TP / (second_TP + second_FN)
second_logistic_predictor_far = second_FP / (second_FP + second_TN)

third_logistic_predictor_precision = third_TP / (third_TP + third_FP)
third_logistic_predictor_recall = third_TP / (third_TP + third_FN)
third_logistic_predictor_far = third_FP / (third_FP + third_TN)

print("First team precision:", first_logistic_predictor_precision)
print("First team recall:", first_logistic_predictor_recall)
print("First team false-alarm:", first_logistic_predictor_far)

print("Second team precision:", second_logistic_predictor_precision)
print("Second team recall:", second_logistic_predictor_recall)
print("Second team false-alarm:", second_logistic_predictor_far)

print("Third team precision:", third_logistic_predictor_precision)
print("Third team recall:", third_logistic_predictor_recall)
print("Third team false-alarm:", third_logistic_predictor_far)
'''